**GRAMMY WINNER PREDICTION**

**PROJET DALAS**

Hiba Akbi, Dwi Prima Putri


**2. SONGS FEATURE DATA SCRAPPING**

In [22]:
import bs4
import lxml
import pandas as pd
import urllib
from bs4 import BeautifulSoup
from urllib import request
import numpy as np
import re
import time

In [ ]:

!pip install beautifulsoup4


Je cree une focntion pour ecrire mes donees dans un fichier.

In [ ]:
import csv
def save_to_file(filename, header, data):
  with open(filename, mode="w", newline="") as file:
      writer = csv.writer(file)
      writer.writerow(header)
      writer.writerows(data)

In [ ]:
#fucntion to read data from a csv file. return a list of lists.
def read_file(filename):
  with open(filename, "r", newline="", encoding="utf-8") as file:
      reader = csv.reader(file)
      header=next(reader)
      data = list(reader)  # Convert to list of rows
  return header, data

The aim here is to get the unique songs that charted (because of course, we have a lot of redudant songs in the chart over the weeks).
Once we get the unique songs, we will go on to extract other features from the songs.


# Find Unique Songs from Billboard

In [ ]:
header, hot_100=read_file("bill_hot100.csv")

In [ ]:
hot_100_df= pd.DataFrame(hot_100, columns=header)

print("original data, we had: ", hot_100_df.shape[0], " song entries.")

unique_songs = hot_100_df.drop_duplicates(subset=['Title', 'Artist'], keep='first')
print("After deleting redundant songs, we have: ", unique_songs.shape[0], " unique songs.")

unique_songs= unique_songs.drop(columns=['\ufeffThe Week', 'Rank','Last Week', 'Peak Pos',
       'Weeks on Chart', 'Award'])

unique_songs

original data, we had:  135800  song entries.
After deleting redundant songs, we have:  11451  unique songs.


,Title,Artist,Song Writer,Producer,Label
0,I Don't Want To Miss A Thing,Aerosmith,D.E.Warren,M.Serletic,Columbia
1,The First Night,Monica,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista
2,Crush,Jennifer Paige,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood
3,My Way,Usher,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista
4,The Boy Is Mine,Brandy & Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic
...,...,...,...,...,...
135760,Passport Junkie,Rod Wave,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo
135773,Talk Talk,Charli xcx Featuring Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic
135774,Bad Dreams,Teddy Swims,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner
135794,I Never Lie,Zach Top,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33


# Split Artist and Featurings

It is a bit of an issue to look for songs using spotify API based on the song title only, instead we need the name of the main artist. In our unique_songs data, the artist column sometimes include artist featuring (ex: Fuerza Regida & Grupo Frontera) or (Santana Featuring Rob Thomas). We will do a simple split to separate between the main artist that we will keep in "Artist" column, and the secondary artists if any, that we will store in "Feat" column. We will use regex, because sometimes the splitting can be tricky. For example, we can have "A & B", "A feat. B", "A featuring B" or even when there are mutliple artists "A & B featurign C" or "A featuring B & C". We also have other variants : "A x B", "A X B", "A With B" ...etc . In any case, we are interested in the main artists only, we will then get the exact other contributers throughthe spotify API if the song is found.


In [ ]:
import numpy as np
import re

pattern = r"\s*(Feat(?:\.|uring)|&|,| X | x |Duet With|With|ft\.| And | Presents | Introducing )\s*"
keywords= ["Featuring", "Feat.", "ft.", "&", "Duet With"," With ", " X ", " x ", ",", " And ", "Presents", "Introducing"]
tests=["Busta Rhymes & Mariah Carey Featuring The Flipmode Squad","Snoop Dogg Presents Tha Eastsidaz","Young Thug And Travis Scott","Bad Bunny X Daddy Yankee","Machine Gun Kelly x Camila Cabello","Prodigy, Illa Ghee & Nina Sky", "Jagged Edge With Nelly", "Jagged Edge With Nelly","Toby Keith Duet With Willie Nelson","Drake","Baha Men With Imani Coppola", "The Roots Featuring Musiq","Fabolous Featuring Mike Shorey & Lil' Mo","Kelly Price & Friends"]
print(tests)

print([re.split(pattern, test, maxsplit=1)[0] if any(keyword in test for keyword in keywords) else test for test in tests ])
print([re.split(pattern, test, maxsplit=1)[2] if any(keyword in test for keyword in keywords) else None for test in tests ])

artist= [re.split(pattern, value, maxsplit=1)[0].strip() if any(k in value for k in keywords) else value for value in unique_songs["Artist"] ]
feats= [re.split(pattern, value, maxsplit=1)[2].strip() if any(k in value for k in keywords) else None for value in unique_songs["Artist"] ]


unique_songs.insert(2, "Main Artist", artist)
unique_songs.insert(3, "Feat", feats)


unique_songs


['Busta Rhymes & Mariah Carey Featuring The Flipmode Squad', 'Snoop Dogg Presents Tha Eastsidaz', 'Young Thug And Travis Scott', 'Bad Bunny X Daddy Yankee', 'Machine Gun Kelly x Camila Cabello', 'Prodigy, Illa Ghee & Nina Sky', 'Jagged Edge With Nelly', 'Jagged Edge With Nelly', 'Toby Keith Duet With Willie Nelson', 'Drake', 'Baha Men With Imani Coppola', 'The Roots Featuring Musiq', "Fabolous Featuring Mike Shorey & Lil' Mo", 'Kelly Price & Friends']
['Busta Rhymes', 'Snoop Dogg', 'Young Thug', 'Bad Bunny', 'Machine Gun Kelly', 'Prodigy', 'Jagged Edge', 'Jagged Edge', 'Toby Keith', 'Drake', 'Baha Men', 'The Roots', 'Fabolous', 'Kelly Price']
['Mariah Carey Featuring The Flipmode Squad', 'Tha Eastsidaz', 'Travis Scott', 'Daddy Yankee', 'Camila Cabello', 'Illa Ghee & Nina Sky', 'Nelly', 'Nelly', 'Willie Nelson', None, 'Imani Coppola', 'Musiq', "Mike Shorey & Lil' Mo", 'Friends']


NameError: name 'unique_songs' is not defined

In [ ]:
unique_songs.iloc[0]

,0
Title,I Don't Want To Miss A Thing
Artist,Aerosmith
Main Artist,Aerosmith
Feat,None
Song Writer,D.E.Warren
Producer,M.Serletic
Label,Columbia


Seems like we are ready now to get some data from spotify API. We are mainly interested in 2 infos:

*  The **Album** the song belongs to
*  The **duration** of the song in seconds.
*  A cleaner version of the **Featured Artists**. Mainly because spotify will include the featured artists, while billboard seem to include all contributing artists.



In [3]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 6.4 MB/s eta 0:00:00


In [9]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up Spotify API credentials
client_id = 'baa1b7896aad4ebabdf13a12fb7da44a'  # Replace with your actual Client ID
client_secret = '451d7b1751274ec9ba0285144a99290f'  # Replace with your actual Client Secret

# Authenticate with Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
import time
album=[]
duration=[]
feats=[]
found=[]
i=0
for idx, row in unique_songs.iterrows():
  i+=1
  # if i>=4783:
  print(i)
  song_name = row["Title"]
  artist_name=row["Main Artist"]
  results = sp.search(q=f"track:{song_name} artist:{artist_name}", limit=1, type='track')
  if idx%2==0: time.sleep(0.2) #to avoid too many request per second
  Flag=1
  # Extract relevant details
  if(len(results['tracks']['items'])==0):
    print("Song not found", row )

    print("-"*10)
    Flag=0
  else: track = results['tracks']['items'][0]

  found.append(Flag) #0 if not found else 1
  album.append(track['album']['name'] if Flag else None)
  duration.append(track['duration_ms']/1000 if Flag else None)
  if Flag:
    all_artists=[artist["name"] for artist in track["artists"]]
    if len(all_artists)==1: feats.append(None) #if theres only one artist, then there is no feat
    else: feats.append(','.join(all_artists[1:])) #store the feats only, not the main artist
  else: feats.append(None)


Streaming output truncated to the last 5000 lines.
7028
7029
7030
7031
7032
7033
7034
7035
7036
7037
7038
7039
7040
7041
7042
7043
7044
7045
7046
7047
7048
7049
7050
7051
7052
7053
7054
7055
7056
7057
7058
7059
Song not found Title          Ghost In This House
Artist                Lauren Duski
Main Artist           Lauren Duski
Feat                          None
Song Writer            H.Prestwood
Producer              B.Appleberry
Label                     Republic
Name: 97597, dtype: object
----------
7060
7061
7062
Song not found Title                                        Deja Vu
Artist                                  Lauren Duski
Main Artist                             Lauren Duski
Feat                                            None
Song Writer    D.C.Tarpley Jr., L.E.Duski, D.D.Henig
Producer                                Cody Tarpley
Label                                       Republic
Name: 97642, dtype: object
----------
7063
7064
7065
Song not found Title                 

In [ ]:
print(len(found))

11451


Our initial run through spotify API with only cleaning the artist name seems quite succesful. We have only 331 missing songs.

In [ ]:
mask=(np.array(found)==0)
filtered_not_found=unique_songs.loc[mask]
print("We have ", len(filtered_not_found), " songs not found out of ", len(unique_songs))

We have  331  songs not found out of  11451


In [ ]:
newdf = unique_songs.copy() #copy to keep this version in case of anything
newdf["Album"]=album
newdf["Duration"]=duration
newdf["Featured Artists"]=feats
newdf["Spotify Found"]=found
newdf

,Title,Artist,Main Artist,Feat,Song Writer,Producer,Label,Album,Duration,Featured Artists,Spotify Found
0,I Don't Want To Miss A Thing,Aerosmith,Aerosmith,None,D.E.Warren,M.Serletic,Columbia,I Don't Want To Miss A Thing EP,299.723,None,1
1,The First Night,Monica,Monica,None,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista,The Boy Is Mine,235.573,None,1
2,Crush,Jennifer Paige,Jennifer Paige,None,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood,Jennifer Paige,200.293,None,1
3,My Way,Usher,Usher,None,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista,My Way,214.853,None,1
4,The Boy Is Mine,Brandy & Monica,Brandy,Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic,the boy is mine (Remix),213.384,"Brandy,Monica",1
...,...,...,...,...,...,...,...,...,...,...,...
135760,Passport Junkie,Rod Wave,Rod Wave,None,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo,Last Lap,147.749,None,1
135773,Talk Talk,Charli xcx Featuring Troye Sivan,Charli xcx,Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic,Brat and it’s completely different but also st...,173.076,Troye Sivan,1
135774,Bad Dreams,Teddy Swims,Teddy Swims,None,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner,Bad Dreams,184.577,None,1
135794,I Never Lie,Zach Top,Zach Top,None,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33,Cold Beer & Country Music,224.906,None,1


Now, we store our unique songs with the additional data we got from spotify in a csv file. Later, we will just import this file to continue our preprocessign to avoid the excessive Spotify API calls for each data loading.

In [ ]:
newdf.to_csv('unique_songs.csv', index=True) #index is holding the ID of the row in the original billbaord data
header, single_songs=read_file("unique_songs.csv")
header=['Song ID', 'Title', 'Artist', 'Main Artist', 'Feat', 'Song Writers/Credits', 'Producer', 'Label', 'Album', 'Duration', 'Spotify Featured Artists', 'Spotify Found']
single_songs_df= pd.DataFrame(single_songs, columns=header)
single_songs_df.to_csv('single_songs.csv', index=False) #fix the name of columns and save it again


**Only piece of code to run for next time**

In [ ]:
header, single_songs=read_file("single_songs.csv")
single_songs_df= pd.DataFrame(single_songs, columns=header)
single_songs_df #11451 x 12

,Song ID,Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,Duration,Spotify Featured Artists,Spotify Found
0,0,I Don't Want To Miss A Thing,Aerosmith,Aerosmith,,D.E.Warren,M.Serletic,Columbia,I Don't Want To Miss A Thing EP,299.723,,1
1,1,The First Night,Monica,Monica,,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista,The Boy Is Mine,235.573,,1
2,2,Crush,Jennifer Paige,Jennifer Paige,,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood,Jennifer Paige,200.293,,1
3,3,My Way,Usher,Usher,,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista,My Way,214.853,,1
4,4,The Boy Is Mine,Brandy & Monica,Brandy,Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic,the boy is mine (Remix),213.384,"Brandy,Monica",1
...,...,...,...,...,...,...,...,...,...,...,...,...
11446,135760,Passport Junkie,Rod Wave,Rod Wave,,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo,Last Lap,147.749,,1
11447,135773,Talk Talk,Charli xcx Featuring Troye Sivan,Charli xcx,Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic,Brat and it’s completely different but also st...,173.076,Troye Sivan,1
11448,135774,Bad Dreams,Teddy Swims,Teddy Swims,,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner,Bad Dreams,184.577,,1
11449,135794,I Never Lie,Zach Top,Zach Top,,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33,Cold Beer & Country Music,224.906,,1



This block of code is just meant to just organzie all of these by idx so that our data remains trackable. So I will assign to each unique song a unique ID (which is its index in the original billbaord data), and I will give the same ID to all its duplicates. This way it will be easy to draw charts on a single song over time. And also merge different tables based on unqiue song ID.


So now, the songs in "hot_100_idx.csv" which is the billbaord data (that contains many duplicates for the same song), and the songs in "unique_songs.csv" can be joined by the column "Song_ID". This will help us later especially for visualization.

In [ ]:
og_df= hot_100_df.copy()
hot_100_idx = og_df.merge(single_songs_df[['Title', 'Artist', 'Song ID']], on=['Title', 'Artist'], how='left')
cols = ['Song ID'] + [col for col in hot_100_idx.columns if col != 'Song ID']
hot_100_idx = hot_100_idx[cols]

# print(hot_100_idx.columns)

hot_100_idx.to_csv('hot_100_idx.csv')

# Data Cleaning to improve Spotify Search

By taking a look at the missing data we have, we notice the following:
* Some songs might not have been found because of **uncleaned details in their title**. For example: Obsession (No Es Amor) having parenthesis for the alternative title. It's worth trying to clean the data that wasn't found to see if the songs name get recognized with only part of the title. There's also a pattern with **swear words** not being recognized, and words containing the **$** sign.

* Some songs weren't recngonized because of the artist name. For exmaple many songs appearing as **"¥$: Kanye West"** would probably be found in the spotify catalog as "Kanye West"


* Some songs with clean titles and artist name might not have been recognized simply because they don not exist in the Spotify Catalog.


Therefore, we will proceed to a little bit more data proprocessing.

* First to handle swear words, we will have a small dictionary of swear words and reconstruct "F*uck" into "Fuck".

* We will manually handle the issue of "Ye". And we will replace the dollar signs by an S isntead.

Lets' run a small tets to tets our hypothesis on songs that were not found earlier.



In [ ]:
test_songs=["All These Niggas", "Give Me A Sign", "Promotion", "Never Too Far" , "Ransom"]
test_artists=["King Von", "Breaking Benjamin", "Kanye West", "Mariah Carey", "Lil Tecca"]

for song_name, artist_name in zip(test_songs, test_artists):
  results = sp.search(q=f"track:{song_name} artist:{artist_name}", limit=1, type='track')
  # Extract relevant details
  if(len(results['tracks']['items'])==0):
    print("Song not found ", song_name, "\t", artist_name )

  else : print("Yay song was found now")

Yay song was found now
Yay song was found now
Yay song was found now
Yay song was found now
Yay song was found now


Seems like the preprocessing is working great, so we apply it.
We will put the cleaned data in a newdf.

# Profanity Decoder
First we build our profanity decoder.
I take a firts look at the type of swear words we have in the title.

In [ ]:
#get all profanities (they contain *)
profanities=[row['Title'] for idx, row in single_songs_df.iterrows() if row['Spotify Found']=='0' and '*' in row['Title'] ]
print(profanities)

['From Her Mama (Mama Got A**)', 'P***ycat', 'Shake That Sh**', "Bitches Ain't S**t", 'Love Me Or Hate Me (F**k You!!!!)', 'J**z In My Pants', 'F*ck You', 'F**k Today', 'F**k You! (Forget You)', 'HYFR (Hell Ya F*****g Right)', 'Rich As F**k', 'F**k That Check Up', 'Dope N****z', 'All These N**gas', 'Constant Bulls**t', 'Switch A N*gga Out', "I'm Dat N***a", 'F**k The Industry Pt. 2']


Looks like we have a small vocabulary of words, the issue is with the different ways that * are placed (B\*tch, B\*\*\*h, B\*\*\*\* ...etc), and variations in the word like plural and singular and -ing. This can be easily solved by matching with 'IN' instead of an exact match of the word.

In [ ]:
profanity_dict={
    "mutha*****": "muthafucka",
    "f*ck": "fuck",
    "f***":"fuck",
    "f**k": "fuck",
    "n**kin":"fuckin",
    "f*****g":"fucking",
    "fuck**g":"fucking",
    "s**t": "shit",
    "sh*t": "shit",
    "sh**":"shit",
    "d@mn": "damn",
    "d*mn": "damn",
    "h*ll": "hell",
    "b*****s":"bitches",
    "b****":"bitch",
    "b*tch": "bitch",
    "b**ch": "bitch",
    "b**tch":"bitch",
    "b***h":"bitch",
    "a**": "ass",
    "p*ssy":"pussy",
    "p***y":"pussy",
    "p****":"pussy",
    "n****":"nigga",
    "n***a":"nigga",
    "n**ga":"nigga",
    "n*gga":"nigga",
    "ni**a":"nigga",
    "j**z": "jizz",

}

#decode profanity : (F*ck -> fuck)
def decode_profanity(title):
  for key,value in profanity_dict.items():
    # print(key in title.lower(), key,"----------", title.lower())
    if key in title.lower():
      title=title.lower()
      title=title.replace(key,value)
  return title

In [ ]:
# tets profanities
no_prof=[decode_profanity(t) for t in profanities]
print(profanities)
print(no_prof)

['From Her Mama (Mama Got A**)', 'P***ycat', 'Shake That Sh**', "Bitches Ain't S**t", 'Love Me Or Hate Me (F**k You!!!!)', 'J**z In My Pants', 'F*ck You', 'F**k Today', 'F**k You! (Forget You)', 'HYFR (Hell Ya F*****g Right)', 'Rich As F**k', 'F**k That Check Up', 'Dope N****z', 'All These N**gas', 'Constant Bulls**t', 'Switch A N*gga Out', "I'm Dat N***a", 'F**k The Industry Pt. 2']
['from her mama (mama got ass)', 'pussycat', 'shake that shit', "bitches ain't shit", 'love me or hate me (fuck you!!!!)', 'jizz in my pants', 'fuck you', 'fuck today', 'fuck you! (forget you)', 'hyfr (hell ya fucking right)', 'rich as fuck', 'fuck that check up', 'dope niggaz', 'all these niggas', 'constant bullshit', 'switch a nigga out', "i'm dat nigga", 'fuck the industry pt. 2']


Looks like ou profanity decoder is working, so now we can proceed to other types of title preprocessing. We got to handle the parenthesis in titles, and cases when the songs has two titles (generally written as Title/Altternative_Title).

We also want to handle the case of the \$ that is recognized by making it S.

In [ ]:
#delete whats in parenthesis : Title(Alt. Title) -> Title
def clean_parenthesis(title):
  parenthesis_pattern= r"\(.*?\)"
  title=re.split(parenthesis_pattern, title,maxsplit=1)
  title=" ".join(title)
  return title.strip()


def preprocess_title(title):
  res=title
  if (",") in res: res=res.replace(',', " ") #handle the commas in a title
  if (".") in res: res=re.sub(r"(?<!\d)\.(?!\d)", " ", res) #handle the dots in a title unless its part of a number
  if ("$") in res: res=res.replace('$','s') #handle $
  if ("(") in res: res=clean_parenthesis(res)
  if ("/") in res: res=res.split("/")[0]
  if ("*") in res: res=decode_profanity(res)
  res=re.sub(r" +", " ", res) #replace any adidtional spaces that we created into a single space
  return res.strip()



In [ ]:
titles_not_found=[row['Title'] for idx, row in single_songs_df.iterrows() if row['Spotify Found']=='0' and '(' in row['Title'] or '$' in row['Title'] ]
titles_cleaned=[preprocess_title(t) for t in titles_not_found ]

print(titles_not_found)
print(titles_cleaned)

['SuperThug (What What)', "Everybody's Free (To Wear Sunscreen)", 'The Christmas Song (Chestnuts Roasting On An Open Fire)', "I'll Fly With You (L'amour Toujours)", 'From Her Mama (Mama Got A**)', "Young'n (Holla Back)", 'Rapture (Tastes So Sweet)', "Tell Me (What's Goin' On)", 'Roll Wit M.V.P. (We Be Like! The La La Song)', 'Simply Being Loved (Somnambulist)', 'Sympathy For The Devil (Remixes)', 'Another Postcard (Chimps)', 'Work It (Reinvention)', 'Obsession (No Es Amor)', 'Helena (So Long & Goodnight)', 'Stay With Me (Brass Bed)', "Kryptonite (I'm On It)", "Oh Yes (aka 'Postman')", "What I've Been Looking For (Reprise)", 'For You I Will (Confidence)', 'Love Me Or Hate Me (F**k You!!!!)', "I'll Be Home For Christmas (Live)", 'The Story (I Was Made For You)', 'Do You Know? (The Ping Pong Song)/Dimelo', "The Chipmunk Song (Christmas Don't Be Late) (2007)", 'Witch Doctor (2007)', 'Leave It All To Me (iCarly Theme Song)', 'Check Yes Juliet (Run Baby Run)', 'Pen & Paper (Something Typical

Seems like our title preprocessing is workign great. Let's apply in a new column called "Cleaned_titles". We're gonna keep it in a seperate column and apply it on all rows (whether they were found in Spotify or not), this is so that the preprocessing can be used for other APIs later.

In [ ]:
clean_single_songs_df=single_songs_df.copy() #keep a copy
cleaned_titles = clean_single_songs_df["Title"].apply(lambda x: preprocess_title(x))
clean_single_songs_df.insert(loc=2, column='Cleaned Title', value=cleaned_titles)
clean_single_songs_df #11451x13

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,Duration,Spotify Featured Artists,Spotify Found
0,0,I Don't Want To Miss A Thing,I Don't Want To Miss A Thing,Aerosmith,Aerosmith,,D.E.Warren,M.Serletic,Columbia,I Don't Want To Miss A Thing EP,299.723,,1
1,1,The First Night,The First Night,Monica,Monica,,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista,The Boy Is Mine,235.573,,1
2,2,Crush,Crush,Jennifer Paige,Jennifer Paige,,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood,Jennifer Paige,200.293,,1
3,3,My Way,My Way,Usher,Usher,,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista,My Way,214.853,,1
4,4,The Boy Is Mine,The Boy Is Mine,Brandy & Monica,Brandy,Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic,the boy is mine (Remix),213.384,"Brandy,Monica",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11446,135760,Passport Junkie,Passport Junkie,Rod Wave,Rod Wave,,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo,Last Lap,147.749,,1
11447,135773,Talk Talk,Talk Talk,Charli xcx Featuring Troye Sivan,Charli xcx,Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic,Brat and it’s completely different but also st...,173.076,Troye Sivan,1
11448,135774,Bad Dreams,Bad Dreams,Teddy Swims,Teddy Swims,,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner,Bad Dreams,184.577,,1
11449,135794,I Never Lie,I Never Lie,Zach Top,Zach Top,,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33,Cold Beer & Country Music,224.906,,1


Perefct, seems like the only remaining titles with * aren't porfanities anyway. So we succesfully decoded all our profanities.

In [ ]:
#after preprocessing
#get all profanities (they contain *)
profanities=[row['Cleaned Title'] for idx, row in clean_single_songs_df.iterrows() if '*' in row['Cleaned Title'] ]
print(profanities)

['H*A*M', '***Flawless', 'Wokeuplikethis*', "Sweet * Honey * Buckiin'"]


Now that the titles are clean, let's proceed to cleanign the artist name.
We have to clean the $ and the issue with Kanye West. Let's see if there's anythign else to clean in terms of artist names.


In [ ]:

unclean_artists=[row['Main Artist'] for idx, row in clean_single_songs_df.iterrows() if row['Spotify Found']=='0' and '*' not in row['Title'] and '(' not in row['Title'] ]
print(unclean_artists)

['Puff Daddy', 'Debelah Morgan', 'Stars On 54: Ultra Nate', 'Pure Sugar', "J'Son", 'Funkmaster Flex', 'Tragedy', 'Tami Davis', 'Missjones', 'Mag 7', 'BRAVO All Stars', 'L.F.O. (Lyte Funky Ones)', 'Tami Davis', 'Randy Travis', 'Silkk The Shocker', 'Divine', 'Randy Travis', 'Youngstown', 'Erykah Badu', 'Garth Brooks as Chris Gaines', 'Randy Travis', 'The Lox', 'Dave Hollister', 'Snoop Dogg', 'Jay-Z', 'Damizza', 'Cash Money Millionaires', 'Mr. C The Slide Man', 'Aaron Lewis Of Staind', 'Iconz', 'Monica', 'Sarina Paris', 'Tim Rushlow', 'Jimmy Cozier', 'Carolyn Dawn Johnson', 'Ludacris', 'Master P', 'Mariah Carey', 'R. Kelly', 'Brian McKnight', 'Jade Anderson', 'Irv Gotti', 'Soluna', 'Yasmeen', 'Tim McGraw', 'Common', 'Rascal Flatts', 'American Idol Finalists', 'Lumidee', 'Kiley Dean', 'M.V.P. (Most Valuable Playas)', 'Scotty Emerick', 'Ryan Duarte', 'Obie Trice', 'Joe Nichols', 'Anita Baker', 'Christina Aguilera', 'R. Kelly', 'Melissa Etheridge', 'American Idol Finalists Season 4', 'Bobby 

This seems  abit more problematic. It seems that for most of the names, they are valid artist names.
It wouldn't make sense to delete numbers and punctuation from artists' names because they wouldn't be recognized otherwise (6ix9ine would become ixine).
Also, there are quite famous artist names in there, so the songs not being found probably has to do with Spotify not having the song in their catalog rather than an issue with the name of the artist.

So we will just stick to our intial hypothesis and clean Ye and $, as well as the parenthesis.

In [ ]:
#preprcoessing on artist names
#replace kanye west name
mask_ye= (clean_single_songs_df["Spotify Found"]=='0') & (clean_single_songs_df["Main Artist"].str.contains("¥\$", na=False))
clean_single_songs_df.loc[mask_ye, "Main Artist"] = "Kanye West"
clean_single_songs_df

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,Duration,Spotify Featured Artists,Spotify Found
0,0,I Don't Want To Miss A Thing,I Don't Want To Miss A Thing,Aerosmith,Aerosmith,,D.E.Warren,M.Serletic,Columbia,I Don't Want To Miss A Thing EP,299.723,,1
1,1,The First Night,The First Night,Monica,Monica,,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista,The Boy Is Mine,235.573,,1
2,2,Crush,Crush,Jennifer Paige,Jennifer Paige,,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood,Jennifer Paige,200.293,,1
3,3,My Way,My Way,Usher,Usher,,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista,My Way,214.853,,1
4,4,The Boy Is Mine,The Boy Is Mine,Brandy & Monica,Brandy,Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic,the boy is mine (Remix),213.384,"Brandy,Monica",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11446,135760,Passport Junkie,Passport Junkie,Rod Wave,Rod Wave,,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo,Last Lap,147.749,,1
11447,135773,Talk Talk,Talk Talk,Charli xcx Featuring Troye Sivan,Charli xcx,Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic,Brat and it’s completely different but also st...,173.076,Troye Sivan,1
11448,135774,Bad Dreams,Bad Dreams,Teddy Swims,Teddy Swims,,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner,Bad Dreams,184.577,,1
11449,135794,I Never Lie,I Never Lie,Zach Top,Zach Top,,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33,Cold Beer & Country Music,224.906,,1


In [ ]:
#fix the $ issue
mask=(clean_single_songs_df["Spotify Found"]=='0') & (clean_single_songs_df["Main Artist"].str.contains(r'\$', na=False))
clean_single_songs_df.loc[mask, "Main Artist"] = clean_single_songs_df.loc[mask, "Main Artist"].replace(r'\$', 's', regex=True)


In [ ]:
#here im fixing this becasue i missed it earlier , some stuff relaing to splitting artist names
mask=(clean_single_songs_df["Spotify Found"]=='0') & (clean_single_songs_df["Main Artist"].str.contains(r'R\. Kelly Or Bow Wow \(', na=False))
clean_single_songs_df.loc[mask, "Main Artist"] = "Bow Wow"

mask= (clean_single_songs_df["Main Artist"].str.contains(r' \+ ', na=False))
clean_single_songs_df.loc[mask, "Main Artist"] = clean_single_songs_df.loc[mask, "Main Artist"].str.split(r" \+ ").str[0]

mask= (clean_single_songs_df["Main Artist"].str.contains(" Co-Starring ", na=False))
clean_single_songs_df.loc[mask, "Main Artist"] = clean_single_songs_df.loc[mask, "Main Artist"].str.split(r" Co-Starring ").str[0]


mask= (clean_single_songs_df["Main Artist"].str.lower().str.contains("vs.?", na=False))
clean_single_songs_df.loc[mask, "Main Artist"] = clean_single_songs_df.loc[mask, "Main Artist"].str.lower().str.split(r"vs.?").str[0].str.title()

mask= (clean_single_songs_df["Main Artist"].str.contains(r'/', na=False))
clean_single_songs_df.loc[mask, "Main Artist"] = clean_single_songs_df.loc[mask, "Main Artist"].str.split("/").str[0]

In [ ]:
mask=(clean_single_songs_df["Main Artist"].str.contains(r'Silk Sonic \(', na=False))
clean_single_songs_df.loc[mask, "Main Artist"] = "Silk Sonic"


mask=((clean_single_songs_df["Main Artist"].str.contains(r'\(', na=False)) & (~clean_single_songs_df["Main Artist"].str.contains(r'\)', na=False)))
clean_single_songs_df.loc[mask, "Main Artist"] =  clean_single_songs_df.loc[mask, "Main Artist"].str.split("\(").str[0]



In [ ]:
clean_single_songs_df

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,Duration,Spotify Featured Artists,Spotify Found
0,0,I Don't Want To Miss A Thing,I Don't Want To Miss A Thing,Aerosmith,Aerosmith,,D.E.Warren,M.Serletic,Columbia,I Don't Want To Miss A Thing EP,299.723,,1
1,1,The First Night,The First Night,Monica,Monica,,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista,The Boy Is Mine,235.573,,1
2,2,Crush,Crush,Jennifer Paige,Jennifer Paige,,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood,Jennifer Paige,200.293,,1
3,3,My Way,My Way,Usher,Usher,,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista,My Way,214.853,,1
4,4,The Boy Is Mine,The Boy Is Mine,Brandy & Monica,Brandy,Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic,the boy is mine (Remix),213.384,"Brandy,Monica",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11446,135760,Passport Junkie,Passport Junkie,Rod Wave,Rod Wave,,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo,Last Lap,147.749,,1
11447,135773,Talk Talk,Talk Talk,Charli xcx Featuring Troye Sivan,Charli xcx,Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic,Brat and it’s completely different but also st...,173.076,Troye Sivan,1
11448,135774,Bad Dreams,Bad Dreams,Teddy Swims,Teddy Swims,,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner,Bad Dreams,184.577,,1
11449,135794,I Never Lie,I Never Lie,Zach Top,Zach Top,,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33,Cold Beer & Country Music,224.906,,1


In [ ]:
#same code as earlier, just now we are working with the cleaned data and also working with only rows that weren't previously found.
i=0
for idx, row in clean_single_songs_df.iterrows():
  if row["Spotify Found"]=='0':
    song_name = row["Cleaned Title"]
    artist_name=row["Main Artist"]
    results = sp.search(q=f"track:{song_name} artist:{artist_name}", limit=1, type='track')
    if idx%2==0: time.sleep(0.2) #to avoid too many request per second
    Flag=1
    # Extract relevant details
    if(len(results['tracks']['items'])==0):
      i+=1
      print("Song not found", row )
      print("-"*10)
      Flag=0
    else: track = results['tracks']['items'][0]

    clean_single_songs_df.loc[idx, "Spotify Found"]= Flag #update the flag
    if Flag:
      clean_single_songs_df.loc[idx, "Album"]=track['album']['name']
      clean_single_songs_df.loc[idx, "Duration"]=track['duration_ms']/1000
      all_artists=[artist["name"] for artist in track["artists"]]
      if len(all_artists)!=1: clean_single_songs_df.loc[idx, "Spotify Featured Artists"]=None #if theres only one artist, then there is no feat
      else: clean_single_songs_df.loc[idx, "Spotify Featured Artists"]=','.join(all_artists[1:]) #store the feats only, not the main artist
print(i)

Song not found Song ID                                                                  31
Title                                                          Come With Me
Cleaned Title                                                  Come With Me
Artist                                      Puff Daddy Featuring Jimmy Page
Main Artist                                                      Puff Daddy
Feat                                                             Jimmy Page
Song Writers/Credits        J.P.Page, R.Plant, J.Bonham, S.J.Combs, M.Curry
Producer                                                              Diddy
Label                                                                  Epic
Album                                                                      
Duration                                                                   
Spotify Featured Artists                                                   
Spotify Found                                                            

In [ ]:
not_found = clean_single_songs_df[clean_single_songs_df["Spotify Found"] == 0].shape[0]
print("Even after preprocessing, we still have ", not_found, " songs not found")

Even after preprocessing, we still have  167  songs not found


Great! We now have only 167 missisng songs instead of 331! So our preprocessing was useful.

In [ ]:
clean_single_songs_df

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,Duration,Spotify Featured Artists,Spotify Found
0,0,I Don't Want To Miss A Thing,I Don't Want To Miss A Thing,Aerosmith,Aerosmith,,D.E.Warren,M.Serletic,Columbia,I Don't Want To Miss A Thing EP,299.723,,1
1,1,The First Night,The First Night,Monica,Monica,,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista,The Boy Is Mine,235.573,,1
2,2,Crush,Crush,Jennifer Paige,Jennifer Paige,,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood,Jennifer Paige,200.293,,1
3,3,My Way,My Way,Usher,Usher,,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista,My Way,214.853,,1
4,4,The Boy Is Mine,The Boy Is Mine,Brandy & Monica,Brandy,Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic,the boy is mine (Remix),213.384,"Brandy,Monica",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11446,135760,Passport Junkie,Passport Junkie,Rod Wave,Rod Wave,,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo,Last Lap,147.749,,1
11447,135773,Talk Talk,Talk Talk,Charli xcx Featuring Troye Sivan,Charli xcx,Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic,Brat and it’s completely different but also st...,173.076,Troye Sivan,1
11448,135774,Bad Dreams,Bad Dreams,Teddy Swims,Teddy Swims,,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner,Bad Dreams,184.577,,1
11449,135794,I Never Lie,I Never Lie,Zach Top,Zach Top,,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33,Cold Beer & Country Music,224.906,,1


We cut down the number of songs from 331 to 167, but we still have some missing songs. However, this will not hinder our work. Let's save this cleaned songs file, then proceed to some more data scrapping.

In [ ]:
clean_single_songs_df.to_csv('clean_single_songs.csv', index=False) #fix the name of columns and save it again

OK, looks good. Now let us scrape the data for the lyrics of the song.

**Only piece of code to run for next time**

Jusqu'ici on a une table qui contient toutes les chansons uniques de billbaord hot 100, sans duplicates.
* **Song ID**: un ID unqiue qu'on a attribue a une chanson. On l'utilisera comme cle primaire pour faire le sjointures sur d'autres tableaux.
* **Title**: Le titre original de la chanson, comme ecrit dans Billboard.
* **Cleaned Title**: La column de titre, mais on a applique une fonction de data cleaning qui decode les insultes, entre autres.
* **Artist**: Le nom du ou des artistes contirbants a la chanson, comme donne par billboard.
* **Main Artist**: L'artiste primaire de la chanson.
* **Feat**: Une version non clean des feats de la chanson, cette column est le resultat du split sur la colomne Main Artist.
* **Song Writers/ Credits**: Les auteurs de la chanson.
* **Producer**: Producteur(s) de la chanson.
* **Label**: Label / maison de disque (?)
* **Album**: Album auquel la chanson appartient. On a recupere ca de Spotify si disponible.
* **Duration**: Duree de la chanson en secondes.
* **Spotify Featured Artists**: Les artistes secondaires de la chanson, version clean comme donnee par Spotify si disponible.
* **Spotify Found**: 1 si a chanson a ete trouvee sur spotify, 0 sinon. Cette column nous aide pour remplir les donees manquantes, elle sera dropped plus tard.

In [ ]:
header, clean_single_songs=read_file("clean_single_songs.csv")
clean_single_songs_df= pd.DataFrame(clean_single_songs, columns=header)
clean_single_songs_df #11451 x 13

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,Duration,Spotify Featured Artists,Spotify Found
0,0,I Don't Want To Miss A Thing,I Don't Want To Miss A Thing,Aerosmith,Aerosmith,,D.E.Warren,M.Serletic,Columbia,I Don't Want To Miss A Thing EP,299.723,,1
1,1,The First Night,The First Night,Monica,Monica,,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista,The Boy Is Mine,235.573,,1
2,2,Crush,Crush,Jennifer Paige,Jennifer Paige,,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood,Jennifer Paige,200.293,,1
3,3,My Way,My Way,Usher,Usher,,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista,My Way,214.853,,1
4,4,The Boy Is Mine,The Boy Is Mine,Brandy & Monica,Brandy,Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic,the boy is mine (Remix),213.384,"Brandy,Monica",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11446,135760,Passport Junkie,Passport Junkie,Rod Wave,Rod Wave,,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo,Last Lap,147.749,,1
11447,135773,Talk Talk,Talk Talk,Charli xcx Featuring Troye Sivan,Charli xcx,Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic,Brat and it’s completely different but also st...,173.076,Troye Sivan,1
11448,135774,Bad Dreams,Bad Dreams,Teddy Swims,Teddy Swims,,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner,Bad Dreams,184.577,,1
11449,135794,I Never Lie,I Never Lie,Zach Top,Zach Top,,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33,Cold Beer & Country Music,224.906,,1


# Scrapping Missing DATA from GENIUS API

We will scrap data from the Genius Api to fill in the gaps of misisng features and  missing values. We will also get the release date of the song, since the available Spotify release date is garbage (sometimes it returned the release date of the song, sometimes of the album).


In [23]:
import bs4
import lxml
import pandas as pd
import urllib
from bs4 import BeautifulSoup
from urllib import request

In [24]:
import requests
from bs4 import BeautifulSoup
import time

In [27]:
import requests
GENIUS_ACCESS_TOKEN = "IvJZGAB3Xav52FL344TG-kajiMHdkVLkDF_e-00qTzJzq0Ts8L-O2aYdUOiyHdBS"
headers = {
    "Authorization": f"Bearer {GENIUS_ACCESS_TOKEN}",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }


We're also going to try and complete the misisng data from spotify real quick.
So we know we have 167 misisng songs , but let's see how many Albums we are missing and how many durations we are missing.

In [ ]:
no_album= clean_single_songs_df[(clean_single_songs_df["Album"]=="") & (clean_single_songs_df["Spotify Found"]=="1") ].shape[0]
print("Out fo the songs found on spotify, we still have ", no_album, " song with NO album data not found")
no_duration= clean_single_songs_df[(clean_single_songs_df["Duration"]=="") & (clean_single_songs_df["Spotify Found"]=="1") ].shape[0]
print("Out fo the songs found on spotify, we still have ", no_duration, " song with NO duration data not found")

Out fo the songs found on spotify, we still have  0  song with NO album data not found
Out fo the songs found on spotify, we still have  0  song with NO duration data not found


Ok, turns out we got data on both features for all songs thta we found, so we are just missing these features for the songs we couldnt find on Spotify.

This isn't a hard fix, it means that if we have a better luck at finding songs usign genius API , we could just extract those features from the genius API to complete the data.

In [ ]:
import time
i=0
genius_found=[]
release_date=[]
album_genius=[]
for idx, row in clean_single_songs_df.iterrows():
  i+=1
  print(i)
  song_name = row["Cleaned Title"]
  artist_name=row["Main Artist"]
  url, id= search_song_GENIUS(song_name, artist_name)
  # if idx%2==0: time.sleep(0.2) #to avoid too many request per second
  Flag=1
  # Extract relevant details
  if(id==None):
    print("Song not found", row )

    print("-"*10)
    Flag=0
  else:  #song was found
    date= get_song_date(id)
    #extract album name of the data if it wasnt found on spotify
    if row["Spotify Found"]=='0' and row['Album']=="":
      print("Genius Found, but spotify didn't. Search for Album of:", song_name," by ", artist_name)
      album= get_song_album(id)
    else: album=None

  genius_found.append(Flag) #0 if not found else 1
  release_date.append(date if Flag else None)
  album_genius.append(album if Flag else None )

In [ ]:
print(len(genius_found))
print(len(release_date))
print(len(album_genius))

11451
11451
11451


Notice again what are the missing songs from the geniusAPI ?


In [ ]:
clean_single_songs_df["Release Date"]=release_date
clean_single_songs_df["Genius Album"]=album_genius
clean_single_songs_df["Genius Found"]=genius_found

In [ ]:
clean_single_songs_df.to_csv('clean_single_songs_genius.csv', index=False)

**Code to run next time**

In [ ]:
header, clean_single_songs=read_file("clean_single_songs_genius.csv")
clean_single_songs_df= pd.DataFrame(clean_single_songs, columns=header)
clean_single_songs_df #11451 x 16

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,Duration,Spotify Featured Artists,Spotify Found,Release Date,Genius Album,Genius Found
0,0,I Don't Want To Miss A Thing,I Don't Want To Miss A Thing,Aerosmith,Aerosmith,,D.E.Warren,M.Serletic,Columbia,I Don't Want To Miss A Thing EP,299.723,,1,1998-08-18,,1
1,1,The First Night,The First Night,Monica,Monica,,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista,The Boy Is Mine,235.573,,1,1998-07-07,,1
2,2,Crush,Crush,Jennifer Paige,Jennifer Paige,,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood,Jennifer Paige,200.293,,1,1998-06-16,,1
3,3,My Way,My Way,Usher,Usher,,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista,My Way,214.853,,1,1997-09-16,,1
4,4,The Boy Is Mine,The Boy Is Mine,Brandy & Monica,Brandy,Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic,the boy is mine (Remix),213.384,"Brandy,Monica",1,1998-05-19,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11446,135760,Passport Junkie,Passport Junkie,Rod Wave,Rod Wave,,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo,Last Lap,147.749,,1,2024-09-14,,1
11447,135773,Talk Talk,Talk Talk,Charli xcx Featuring Troye Sivan,Charli xcx,Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic,Brat and it’s completely different but also st...,173.076,Troye Sivan,1,2024-06-07,,1
11448,135774,Bad Dreams,Bad Dreams,Teddy Swims,Teddy Swims,,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner,Bad Dreams,184.577,,1,2024-09-13,,1
11449,135794,I Never Lie,I Never Lie,Zach Top,Zach Top,,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33,Cold Beer & Country Music,224.906,,1,2024-04-05,,1


In [ ]:
#
df= clean_single_songs_df[clean_single_songs_df["Genius Found"]=='0']
print("We weren't able to find : ",len(df), " songs using Genius API")


We weren't able to find :  6  songs using Genius API


Akright, looks like only 6 songs couldnt be found on geniusAPI. What interests us thought is the numbe rof songs for which we dont have releas dates. So let's see that.

In [ ]:
df = clean_single_songs_df[clean_single_songs_df["Release Date"]==""]
print("We are missing the release dates of : ",len(df), " songs using Genius API")


We are missing the release dates of :  621  songs using Genius API


Let's see if we filled the gaps of the album release


In [ ]:
df = clean_single_songs_df[(clean_single_songs_df["Genius Album"]!="") & (clean_single_songs_df["Album"]=="")]
print("We filled the album gap of : ",len(df), " songs using Genius API")

We filled the album gap of :  129  songs using Genius API


In [ ]:
df

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,Duration,Spotify Featured Artists,Spotify Found,Release Date,Genius Album,Genius Found
31,31,Come With Me,Come With Me,Puff Daddy Featuring Jimmy Page,Puff Daddy,Jimmy Page,"J.P.Page, R.Plant, J.Bonham, S.J.Combs, M.Curry",Diddy,Epic,,,,0,1998-05-19,Godzilla: The Album,1
65,65,If You Could Read My Mind,If You Could Read My Mind,"Stars On 54: Ultra Nate, Amber, Jocelyn Enriquez",Stars On 54: Ultra Nate,"Amber, Jocelyn Enriquez",G.Lightfoot,The Berman Brothers,Tommy Boy,,,,0,,"54 (Music From the Miramax Motion Picture, Vol...",1
67,67,Delicious,Delicious,Pure Sugar,Pure Sugar,,"P.Lorimer, R.Vission, J.Johnson, Kibble, M.Bra...","P.Lorimer, R.Vission",Geffen,,,,0,2017-01-24,Pure Comedy,1
80,80,Here We Go,Here We Go,Funkmaster Flex Presents Khadejia Featuring Pr...,Funkmaster Flex,Khadejia Featuring Product,"W.Jean, K.Bass, D.McRae, M.Moore-Hough","W.Jean, Funkmaster Flex",Loud/RCA,,,,0,1998-08-11,"The Mix Tape Volume III: 60 Minutes of Funk, T...",1
85,85,'98 Thug Paradise,'98 Thug Paradise,"Tragedy, Capone, Infinite",Tragedy,"Capone, Infinite","A.Moody, W.Hale","A.McLean, D-Moet",H.O.L.A.,,,,0,,The Wretched Of The Earth,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9951,123309,I'm Dat N***a,i'm dat nigga,Future,Future,,"N.D.Cash, D.L.J.Avery, N.Santos, J.H.Luellen","DY Krazy, Nicky Slowburnz, J.Luellen",Freebandz/Epic,,,,0,2022-04-29,I NEVER LIKED YOU (EXTENDED),1
10092,124175,Call Me Everyday,Call Me Everyday,Chris Brown Featuring WizKid,Chris Brown,WizKid,"C.M.Brown, M.Adeyinka, M.Akunwata, L.D.Youngbl...","Blaise Beatz, Roccstar",CBE/RCA,,,,0,2022-06-17,Breezy (Deluxe),1
10997,132230,Pop Ur Sht,Pop Ur Sht,"21 Savage, Young Thug & Metro Boomin",21 Savage,Young Thug & Metro Boomin,"S.B.A.Joseph, J.L.Williams, L.T.Wayne, A.Ritter","Metro Boomin, A.Ritter",Slaughter Gang/Epic,,,,0,,things,1
11152,133405,7 Minute Drill,7 Minute Drill,J. Cole,J. Cole,,J.Cole,"T-Minus, Conductor Williams",Cole World/Interscope/ICLG,,,,0,2024-04-05,Might Delete Later,1


That's  great, the only annoying part is our 600 missing release dates, we will see how to fix this using Wikipedia. For now, we save our data.


**WHAT TO RUN NEXT TIME**

In [ ]:
header, clean_single_songs=read_file("clean_single_songs_genius.csv")
clean_single_songs_df= pd.DataFrame(clean_single_songs, columns=header)
clean_single_songs_df #11451 x 16

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,Duration,Spotify Featured Artists,Spotify Found,Release Date,Genius Album,Genius Found
0,0,I Don't Want To Miss A Thing,I Don't Want To Miss A Thing,Aerosmith,Aerosmith,,D.E.Warren,M.Serletic,Columbia,I Don't Want To Miss A Thing EP,299.723,,1,1998-08-18,,1
1,1,The First Night,The First Night,Monica,Monica,,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista,The Boy Is Mine,235.573,,1,1998-07-07,,1
2,2,Crush,Crush,Jennifer Paige,Jennifer Paige,,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood,Jennifer Paige,200.293,,1,1998-06-16,,1
3,3,My Way,My Way,Usher,Usher,,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista,My Way,214.853,,1,1997-09-16,,1
4,4,The Boy Is Mine,The Boy Is Mine,Brandy & Monica,Brandy,Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic,the boy is mine (Remix),213.384,"Brandy,Monica",1,1998-05-19,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11446,135760,Passport Junkie,Passport Junkie,Rod Wave,Rod Wave,,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo,Last Lap,147.749,,1,2024-09-14,,1
11447,135773,Talk Talk,Talk Talk,Charli xcx Featuring Troye Sivan,Charli xcx,Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic,Brat and it’s completely different but also st...,173.076,Troye Sivan,1,2024-06-07,,1
11448,135774,Bad Dreams,Bad Dreams,Teddy Swims,Teddy Swims,,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner,Bad Dreams,184.577,,1,2024-09-13,,1
11449,135794,I Never Lie,I Never Lie,Zach Top,Zach Top,,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33,Cold Beer & Country Music,224.906,,1,2024-04-05,,1


# Scrapping the Lyrics

Now that we have some better cleaned artist name and song titles, and that we have set up GENIUS API, we will proceed to get the lyrics of the songs.

In [20]:
def get_lyrics(song, artist):
    # Construct the URL
    url = f"https://api.lyrics.ovh/v1/{artist}/{song}"

    # Send the request
    response = requests.get(url)

    # Check if the response is valid
    if response.status_code == 200:
        data = response.json()
        # Return the lyrics from the response
        return data.get('lyrics', None)
    else:
        return None

    return None




In [ ]:
lyrics= get_lyrics("I Don't Want To Miss A Thing","Aerosmith")
print(lyrics)

I could stay awake just to hear you breathing
Watch you smile while you are sleeping
While you're far away and dreaming
I could spend my life in this sweet surrender
I could stay lost in this moment forever
Every moment spent with you is a moment I treasure



Don't wanna close my eyes

I don't wanna fall asleep

'Coz I'd miss you baby

And I don't wanna miss a thing

'Coz even when I dream of you

The sweetest dream would never do

I'd still miss you baby

And I don't wanna miss a thing



Laying close to you

Feeling your heart beating

And I'm wondering what you're dreaming

Wondering if it's me you're seeing

Then I kiss your eyes

And thank God we're together

I just want to stay with you in this moment forever

Forever and ever



Don't wanna close my eyes

I don't wanna fall asleep

'Coz I'd miss you baby

And I don't wanna miss a thing

'Coz even when I dream of you

The sweetest dream would never do

I'd still miss you baby

And I don't wanna miss a thing



I don't wanna miss

In [ ]:
import time
i=0
lyrics_all=[]
for idx, row in clean_single_songs_df.iterrows():
  i+=1
  print(i)
  song_name = row["Cleaned Title"]
  artist_name=row["Main Artist"]
  # idx%2==0: time.sleep(0.2) #slow down
  lyrics=get_lyrics(song_name, artist_name)
  if lyrics==None: print("Couln't find lyric for: ", artist_name, song_name)
  lyrics_all.append(lyrics)


Streaming output truncated to the last 5000 lines.
7120
Couln't find lyric for:  JAY-Z Kill Jay Z
7121
7122
Couln't find lyric for:  JAY-Z Caught Their Eyes
7123
Couln't find lyric for:  JAY-Z Moonlight
7124
7125
7126
7127
7128
Couln't find lyric for:  21 Savage Famous
7129
7130
7131
Couln't find lyric for:  Khalid Young Dumb & Broke
7132
7133
7134
7135
7136
Couln't find lyric for:  Meek Mill Wins & Losses
7137
Couln't find lyric for:  Descendants 2 Cast It's Goin' Down
7138
7139
7140
7141
7142
7143
7144
Couln't find lyric for:  Thomas Rhett Unforgettable
7145
7146
7147
7148
7149
7150
7151
7152
Couln't find lyric for:  Kesha Learn To Let Go
7153
7154
7155
7156
7157
7158
Couln't find lyric for:  P!nk What About Us
7159
7160
Couln't find lyric for:  Jacquees B E D
7161
Couln't find lyric for:  Sofia Carson Chillin' Like A Villain
7162
7163
7164
7165
7166
7167
7168
7169
7170
7171
7172
7173
7174
Couln't find lyric for:  RiceGum Its Every Night Sis
7175
7176
7177
7178
Couln't find lyric for

Regardons combien de lyrics on n'a pas pu trouver

In [ ]:
clean_single_songs_df["Lyrics"]= lyrics_all

In [ ]:
clean_single_songs_df

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,Duration,Spotify Featured Artists,Spotify Found,Release Date,Genius Album,Genius Found,Lyrics
0,0,I Don't Want To Miss A Thing,I Don't Want To Miss A Thing,Aerosmith,Aerosmith,,D.E.Warren,M.Serletic,Columbia,I Don't Want To Miss A Thing EP,299.723,,1,1998-08-18,,1,I could stay awake just to hear you breathing\...
1,1,The First Night,The First Night,Monica,Monica,,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista,The Boy Is Mine,235.573,,1,1998-07-07,,1,[JD:]\r\nBaby baby tell me whats up? Can you h...
2,2,Crush,Crush,Jennifer Paige,Jennifer Paige,,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood,Jennifer Paige,200.293,,1,1998-06-16,,1,See ya blowin' me a kiss\r\nIt doesn't take a ...
3,3,My Way,My Way,Usher,Usher,,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista,My Way,214.853,,1,1997-09-16,,1,"Ooh, Ooh, Hey Yo see it's rare that you find p..."
4,4,The Boy Is Mine,The Boy Is Mine,Brandy & Monica,Brandy,Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic,the boy is mine (Remix),213.384,"Brandy,Monica",1,1998-05-19,,1,"Monica \nExcuse me do I know you cause uhh, yo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11446,135760,Passport Junkie,Passport Junkie,Rod Wave,Rod Wave,,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo,Last Lap,147.749,,1,2024-09-14,,1,Bahamas might work\r\nTurks might work\r\nI co...
11447,135773,Talk Talk,Talk Talk,Charli xcx Featuring Troye Sivan,Charli xcx,Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic,Brat and it’s completely different but also st...,173.076,Troye Sivan,1,2024-06-07,,1,I've been lookin' at you\r\nPuttin' holes in y...
11448,135774,Bad Dreams,Bad Dreams,Teddy Swims,Teddy Swims,,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner,Bad Dreams,184.577,,1,2024-09-13,,1,Sun is goin' down time is runnin' out\r\nNo on...
11449,135794,I Never Lie,I Never Lie,Zach Top,Zach Top,,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33,Cold Beer & Country Music,224.906,,1,2024-04-05,,1,"Well, it's been some time\r\nYou still look li..."


Great, we got our songs, let's see how much we are missing.


In [ ]:
df= clean_single_songs_df[clean_single_songs_df["Lyrics"].isna()]
print(f"We are misisng lyrics for {len(df)} songs")

We are misisng lyrics for 1764 songs


We have 1764 songs that we coulnd't get lyrics for. Let's see if we can fix that.

We will ge tthe songs from another platform.

In [ ]:
clean_single_songs_df.to_csv('clean_single_songs_genius_lyrics.csv', index=False) #fix the name of columns and save it again

## whats to run next time

In [ ]:
import pandas as pd

songs_df = pd.read_csv("clean_single_songs_genius_lyrics.csv")

In [ ]:
songs_df

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,Duration,Spotify Featured Artists,Spotify Found,Release Date,Genius Album,Genius Found,Lyrics
0,0,I Don't Want To Miss A Thing,I Don't Want To Miss A Thing,Aerosmith,Aerosmith,NaN,D.E.Warren,M.Serletic,Columbia,I Don't Want To Miss A Thing EP,299.723,NaN,1,1998-08-18,NaN,1,I could stay awake just to hear you breathing\...
1,1,The First Night,The First Night,Monica,Monica,NaN,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista,The Boy Is Mine,235.573,NaN,1,1998-07-07,NaN,1,[JD:]\r\nBaby baby tell me whats up? Can you h...
2,2,Crush,Crush,Jennifer Paige,Jennifer Paige,NaN,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood,Jennifer Paige,200.293,NaN,1,1998-06-16,NaN,1,See ya blowin' me a kiss\r\nIt doesn't take a ...
3,3,My Way,My Way,Usher,Usher,NaN,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista,My Way,214.853,NaN,1,1997-09-16,NaN,1,"Ooh, Ooh, Hey Yo see it's rare that you find p..."
4,4,The Boy Is Mine,The Boy Is Mine,Brandy & Monica,Brandy,Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic,the boy is mine (Remix),213.384,"Brandy,Monica",1,1998-05-19,NaN,1,"Monica \nExcuse me do I know you cause uhh, yo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11446,135760,Passport Junkie,Passport Junkie,Rod Wave,Rod Wave,NaN,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo,Last Lap,147.749,NaN,1,2024-09-14,NaN,1,Bahamas might work\r\nTurks might work\r\nI co...
11447,135773,Talk Talk,Talk Talk,Charli xcx Featuring Troye Sivan,Charli xcx,Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic,Brat and it’s completely different but also st...,173.076,Troye Sivan,1,2024-06-07,NaN,1,I've been lookin' at you\r\nPuttin' holes in y...
11448,135774,Bad Dreams,Bad Dreams,Teddy Swims,Teddy Swims,NaN,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner,Bad Dreams,184.577,NaN,1,2024-09-13,NaN,1,Sun is goin' down time is runnin' out\r\nNo on...
11449,135794,I Never Lie,I Never Lie,Zach Top,Zach Top,NaN,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33,Cold Beer & Country Music,224.906,NaN,1,2024-04-05,NaN,1,"Well, it's been some time\r\nYou still look li..."


In [ ]:
songs_df = songs_df.where(pd.notna(songs_df), None)
songs_df

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,Duration,Spotify Featured Artists,Spotify Found,Release Date,Genius Album,Genius Found,Lyrics
0,0,I Don't Want To Miss A Thing,I Don't Want To Miss A Thing,Aerosmith,Aerosmith,None,D.E.Warren,M.Serletic,Columbia,I Don't Want To Miss A Thing EP,299.723,None,1,1998-08-18,None,1,I could stay awake just to hear you breathing\...
1,1,The First Night,The First Night,Monica,Monica,None,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista,The Boy Is Mine,235.573,None,1,1998-07-07,None,1,[JD:]\r\nBaby baby tell me whats up? Can you h...
2,2,Crush,Crush,Jennifer Paige,Jennifer Paige,None,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood,Jennifer Paige,200.293,None,1,1998-06-16,None,1,See ya blowin' me a kiss\r\nIt doesn't take a ...
3,3,My Way,My Way,Usher,Usher,None,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista,My Way,214.853,None,1,1997-09-16,None,1,"Ooh, Ooh, Hey Yo see it's rare that you find p..."
4,4,The Boy Is Mine,The Boy Is Mine,Brandy & Monica,Brandy,Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic,the boy is mine (Remix),213.384,"Brandy,Monica",1,1998-05-19,None,1,"Monica \nExcuse me do I know you cause uhh, yo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11446,135760,Passport Junkie,Passport Junkie,Rod Wave,Rod Wave,None,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo,Last Lap,147.749,None,1,2024-09-14,None,1,Bahamas might work\r\nTurks might work\r\nI co...
11447,135773,Talk Talk,Talk Talk,Charli xcx Featuring Troye Sivan,Charli xcx,Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic,Brat and it’s completely different but also st...,173.076,Troye Sivan,1,2024-06-07,None,1,I've been lookin' at you\r\nPuttin' holes in y...
11448,135774,Bad Dreams,Bad Dreams,Teddy Swims,Teddy Swims,None,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner,Bad Dreams,184.577,None,1,2024-09-13,None,1,Sun is goin' down time is runnin' out\r\nNo on...
11449,135794,I Never Lie,I Never Lie,Zach Top,Zach Top,None,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33,Cold Beer & Country Music,224.906,None,1,2024-04-05,None,1,"Well, it's been some time\r\nYou still look li..."


In [ ]:
songs_df["Song ID"] = songs_df["Song ID"].astype(str).apply(lambda x: "S" + x)
songs_df

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,Duration,Spotify Featured Artists,Spotify Found,Release Date,Genius Album,Genius Found,Lyrics
0,S0,I Don't Want To Miss A Thing,I Don't Want To Miss A Thing,Aerosmith,Aerosmith,None,D.E.Warren,M.Serletic,Columbia,I Don't Want To Miss A Thing EP,299.723,None,1,1998-08-18,None,1,I could stay awake just to hear you breathing\...
1,S1,The First Night,The First Night,Monica,Monica,None,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista,The Boy Is Mine,235.573,None,1,1998-07-07,None,1,[JD:]\r\nBaby baby tell me whats up? Can you h...
2,S2,Crush,Crush,Jennifer Paige,Jennifer Paige,None,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood,Jennifer Paige,200.293,None,1,1998-06-16,None,1,See ya blowin' me a kiss\r\nIt doesn't take a ...
3,S3,My Way,My Way,Usher,Usher,None,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista,My Way,214.853,None,1,1997-09-16,None,1,"Ooh, Ooh, Hey Yo see it's rare that you find p..."
4,S4,The Boy Is Mine,The Boy Is Mine,Brandy & Monica,Brandy,Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic,the boy is mine (Remix),213.384,"Brandy,Monica",1,1998-05-19,None,1,"Monica \nExcuse me do I know you cause uhh, yo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11446,S135760,Passport Junkie,Passport Junkie,Rod Wave,Rod Wave,None,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo,Last Lap,147.749,None,1,2024-09-14,None,1,Bahamas might work\r\nTurks might work\r\nI co...
11447,S135773,Talk Talk,Talk Talk,Charli xcx Featuring Troye Sivan,Charli xcx,Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic,Brat and it’s completely different but also st...,173.076,Troye Sivan,1,2024-06-07,None,1,I've been lookin' at you\r\nPuttin' holes in y...
11448,S135774,Bad Dreams,Bad Dreams,Teddy Swims,Teddy Swims,None,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner,Bad Dreams,184.577,None,1,2024-09-13,None,1,Sun is goin' down time is runnin' out\r\nNo on...
11449,S135794,I Never Lie,I Never Lie,Zach Top,Zach Top,None,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33,Cold Beer & Country Music,224.906,None,1,2024-04-05,None,1,"Well, it's been some time\r\nYou still look li..."


In [ ]:
songs_df.apply(pd.Series.unique)

,0
Song ID,"[S0, S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, ..."
Title,"[I Don't Want To Miss A Thing, The First Night..."
Cleaned Title,"[I Don't Want To Miss A Thing, The First Night..."
Artist,"[Aerosmith, Monica, Jennifer Paige, Usher, Bra..."
Main Artist,"[Aerosmith, Monica, Jennifer Paige, Usher, Bra..."
Feat,"[None, Monica, Puff Daddy, Missy ""Misdemeanor""..."
Song Writers/Credits,"[D.E.Warren, J.Dupri Mauldin, T.Savage, M.McLe..."
Producer,"[M.Serletic, J.Dupri, A.Goldmark, J.Bralower, ..."
Label,"[Columbia, Arista, edel/Hollywood, LaFace/Aris..."
Album,"[I Don't Want To Miss A Thing EP, The Boy Is M..."


In [ ]:
temp= pd.read_csv("clean_single_songs_DWI_merge_OTHER_FEATURES.csv")
temp


,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,Duration,Spotify Featured Artists,Spotify Found,Genre,Tempo,Loudness
0,0,I Don't Want To Miss A Thing,I Don't Want To Miss A Thing,Aerosmith,Aerosmith,NaN,D.E.Warren,M.Serletic,Columbia,I Don't Want To Miss A Thing EP,299.723,NaN,1,Rock,133.256699,0.944479
1,1,The First Night,The First Night,Monica,Monica,NaN,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista,The Boy Is Mine,235.573,NaN,1,R&B,152.175323,0.919360
2,2,Crush,Crush,Jennifer Paige,Jennifer Paige,NaN,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood,Jennifer Paige,200.293,NaN,1,Pop,115.030586,0.918512
3,3,My Way,My Way,Usher,Usher,NaN,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista,My Way,214.853,NaN,1,Pop,90.065628,0.866996
4,4,The Boy Is Mine,The Boy Is Mine,Brandy & Monica,Brandy,Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic,the boy is mine (Remix),213.384,"Brandy,Monica",1,Pop,93.010178,0.911099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11446,135760,Passport Junkie,Passport Junkie,Rod Wave,Rod Wave,NaN,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo,Last Lap,147.749,NaN,1,Rap/Hip Hop,NaN,NaN
11447,135773,Talk Talk,Talk Talk,Charli xcx Featuring Troye Sivan,Charli xcx,Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic,Brat and it’s completely different but also st...,173.076,Troye Sivan,1,Pop,122.336159,0.844509
11448,135774,Bad Dreams,Bad Dreams,Teddy Swims,Teddy Swims,NaN,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner,Bad Dreams,184.577,NaN,1,Pop,NaN,NaN
11449,135794,I Never Lie,I Never Lie,Zach Top,Zach Top,NaN,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33,Cold Beer & Country Music,224.906,NaN,1,Country,NaN,NaN


In [ ]:
songs_df["Genre"]= temp["Genre"]
songs_df["Tempo"]= temp["Tempo"]
songs_df["Loudness"]= temp["Loudness"]


In [ ]:
songs_df = songs_df.where(pd.notna(songs_df), None)
songs_df

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,Duration,Spotify Featured Artists,Spotify Found,Release Date,Genius Album,Genius Found,Lyrics,Genre,Tempo,Loudness
0,S0,I Don't Want To Miss A Thing,I Don't Want To Miss A Thing,Aerosmith,Aerosmith,None,D.E.Warren,M.Serletic,Columbia,I Don't Want To Miss A Thing EP,299.723,None,1,1998-08-18,None,1,I could stay awake just to hear you breathing\...,Rock,133.256699,0.944479
1,S1,The First Night,The First Night,Monica,Monica,None,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista,The Boy Is Mine,235.573,None,1,1998-07-07,None,1,[JD:]\r\nBaby baby tell me whats up? Can you h...,R&B,152.175323,0.919360
2,S2,Crush,Crush,Jennifer Paige,Jennifer Paige,None,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood,Jennifer Paige,200.293,None,1,1998-06-16,None,1,See ya blowin' me a kiss\r\nIt doesn't take a ...,Pop,115.030586,0.918512
3,S3,My Way,My Way,Usher,Usher,None,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista,My Way,214.853,None,1,1997-09-16,None,1,"Ooh, Ooh, Hey Yo see it's rare that you find p...",Pop,90.065628,0.866996
4,S4,The Boy Is Mine,The Boy Is Mine,Brandy & Monica,Brandy,Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic,the boy is mine (Remix),213.384,"Brandy,Monica",1,1998-05-19,None,1,"Monica \nExcuse me do I know you cause uhh, yo...",Pop,93.010178,0.911099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11446,S135760,Passport Junkie,Passport Junkie,Rod Wave,Rod Wave,None,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo,Last Lap,147.749,None,1,2024-09-14,None,1,Bahamas might work\r\nTurks might work\r\nI co...,Rap/Hip Hop,NaN,NaN
11447,S135773,Talk Talk,Talk Talk,Charli xcx Featuring Troye Sivan,Charli xcx,Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic,Brat and it’s completely different but also st...,173.076,Troye Sivan,1,2024-06-07,None,1,I've been lookin' at you\r\nPuttin' holes in y...,Pop,122.336159,0.844509
11448,S135774,Bad Dreams,Bad Dreams,Teddy Swims,Teddy Swims,None,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner,Bad Dreams,184.577,None,1,2024-09-13,None,1,Sun is goin' down time is runnin' out\r\nNo on...,Pop,NaN,NaN
11449,S135794,I Never Lie,I Never Lie,Zach Top,Zach Top,None,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33,Cold Beer & Country Music,224.906,None,1,2024-04-05,None,1,"Well, it's been some time\r\nYou still look li...",Country,NaN,NaN


In [ ]:
print("None values Lyrics", songs_df["Lyrics"].isna().sum())
print("None values Duration", songs_df["Duration"].isna().sum())
print("None values date", songs_df["Release Date"].isna().sum())
print("None values loudness", songs_df["Loudness"].isna().sum())
print("None values tempo", songs_df["Tempo"].isna().sum())
print("None values genre", songs_df["Genre"].isna().sum())

None values Lyrics 1764
None values Duration 167
None values date 621
None values loudness 1117
None values tempo 1117
None values genre 103


In [ ]:
songs_df.to_csv('songs_data_final.csv', index=False)

# Extra grammys songs

### sptofy search

So from grammys we found some extra songs that werent on billbaord. we will add them to our songs_data.

In [6]:
import pandas as pd

In [17]:
extra_songs_df= pd.read_csv("missing_songs_grammy_dwi.csv")
extra_songs_df

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,...,Spotify Found,Release Date,Genius Album,Genius Found,Lyrics,Genre,Tempo,Loudness,Cleaned Main Artist,ArtistID
0,NaN,Asteroids,Asteroids,Rapsody Featuring Hit-Boy,Rapsody,Hit-Boy,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Rap/Hip Hop,125.890709,0.886106,Rapsody,NaN
1,NaN,Not Strong Enough,Not Strong Enough,boygenius,Boygenius,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Alternative,127.000000,NaN,Boygenius,NaN
2,NaN,Worship,Worship,Jon Batiste,Jon Batiste,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Jazz,97.000000,NaN,Jon Batiste,NaN
3,NaN,Butterfly,Butterfly,Jon Batiste,Jon Batiste,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Pop,140.000000,NaN,Jon Batiste,NaN
4,NaN,A&W,A&W,Lana Del Rey,Lana Del Rey,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Rap/Hip Hop,162.782684,0.900911,Lana Del Rey,NaN
5,NaN,SCIENTISTS & ENGINEERS,Scientists & Engineers,"Killer Mike Featuring André 3000, Future And E...",Killer Mike,"André 3000, Future And Eryn Allen Kane",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Rap/Hip Hop,140.000000,NaN,Killer Mike,NaN
6,NaN,You And Me On The Rock,You And Me On The Rock,Brandi Carlile Featuring Lucius,Brandi Carlile,Lucius,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Singer & Songwriter,113.995728,0.853343,Brandi Carlile,NaN
7,NaN,Don't Shut Me Down,Don'T Shut Me Down,ABBA,Abba,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Pop,99.925713,0.637480,Abba,NaN
8,NaN,Just Like That,Just Like That,Bonnie Raitt,Bonnie Raitt,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Rock,81.000000,NaN,Bonnie Raitt,NaN
9,NaN,Right On Time,Right On Time,Brandi Carlile,Brandi Carlile,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Singer & Songwriter,126.191765,0.469051,Brandi Carlile,NaN


scrap the rest fo the data for these songs.

In [10]:
import time
album=[]
duration=[]
feats=[]
found=[]
i=0
for idx, row in extra_songs_df.iterrows():
  i+=1
  print(i)
  song_name = row["Cleaned Title"]
  artist_name=row["Main Artist"]
  results = sp.search(q=f"track:{song_name} artist:{artist_name}", limit=1, type='track')
  if idx%2==0: time.sleep(0.2) #to avoid too many request per second
  Flag=1
  # Extract relevant details
  if(len(results['tracks']['items'])==0):
    print("Song not found", row )

    print("-"*10)
    Flag=0
  else: track = results['tracks']['items'][0]

  found.append(Flag) #0 if not found else 1
  album.append(track['album']['name'] if Flag else None)
  duration.append(track['duration_ms']/1000 if Flag else None)
  if Flag:
    all_artists=[artist["name"] for artist in track["artists"]]
    if len(all_artists)==1: feats.append(None) #if theres only one artist, then there is no feat
    else: feats.append(','.join(all_artists[1:])) #store the feats only, not the main artist
  else: feats.append(None)



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [11]:
print(len(album)) #should be 42
print(len(duration))
print(len(feats))
print(len(found))

42
42
42
42


In [18]:
extra_songs_df['Album']=album
extra_songs_df['Duration']=duration
extra_songs_df['Album']=album
extra_songs_df['Spotify Featured Artists']=feats
extra_songs_df['Spotify Found']=found
extra_songs_df#42x23

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,...,Spotify Found,Release Date,Genius Album,Genius Found,Lyrics,Genre,Tempo,Loudness,Cleaned Main Artist,ArtistID
0,NaN,Asteroids,Asteroids,Rapsody Featuring Hit-Boy,Rapsody,Hit-Boy,NaN,NaN,NaN,Please Don't Cry,...,1,NaN,NaN,NaN,NaN,Rap/Hip Hop,125.890709,0.886106,Rapsody,NaN
1,NaN,Not Strong Enough,Not Strong Enough,boygenius,Boygenius,NaN,NaN,NaN,NaN,the record,...,1,NaN,NaN,NaN,NaN,Alternative,127.000000,NaN,Boygenius,NaN
2,NaN,Worship,Worship,Jon Batiste,Jon Batiste,NaN,NaN,NaN,NaN,World Music Radio,...,1,NaN,NaN,NaN,NaN,Jazz,97.000000,NaN,Jon Batiste,NaN
3,NaN,Butterfly,Butterfly,Jon Batiste,Jon Batiste,NaN,NaN,NaN,NaN,World Music Radio,...,1,NaN,NaN,NaN,NaN,Pop,140.000000,NaN,Jon Batiste,NaN
4,NaN,A&W,A&W,Lana Del Rey,Lana Del Rey,NaN,NaN,NaN,NaN,Did you know that there's a tunnel under Ocean...,...,1,NaN,NaN,NaN,NaN,Rap/Hip Hop,162.782684,0.900911,Lana Del Rey,NaN
5,NaN,SCIENTISTS & ENGINEERS,Scientists & Engineers,"Killer Mike Featuring André 3000, Future And E...",Killer Mike,"André 3000, Future And Eryn Allen Kane",NaN,NaN,NaN,MICHAEL,...,1,NaN,NaN,NaN,NaN,Rap/Hip Hop,140.000000,NaN,Killer Mike,NaN
6,NaN,You And Me On The Rock,You And Me On The Rock,Brandi Carlile Featuring Lucius,Brandi Carlile,Lucius,NaN,NaN,NaN,In These Silent Days,...,1,NaN,NaN,NaN,NaN,Singer & Songwriter,113.995728,0.853343,Brandi Carlile,NaN
7,NaN,Don't Shut Me Down,Don'T Shut Me Down,ABBA,Abba,NaN,NaN,NaN,NaN,I Still Have Faith In You / Don't Shut Me Down,...,1,NaN,NaN,NaN,NaN,Pop,99.925713,0.637480,Abba,NaN
8,NaN,Just Like That,Just Like That,Bonnie Raitt,Bonnie Raitt,NaN,NaN,NaN,NaN,Just Like That...,...,1,NaN,NaN,NaN,NaN,Rock,81.000000,NaN,Bonnie Raitt,NaN
9,NaN,Right On Time,Right On Time,Brandi Carlile,Brandi Carlile,NaN,NaN,NaN,NaN,Right on Time,...,1,NaN,NaN,NaN,NaN,Singer & Songwriter,126.191765,0.469051,Brandi Carlile,NaN


In [ ]:
# temp= extra_songs_df.drop(["Song Writers/Credits","Producer","Label", "Genius Found", "Lyrics", "Genius Album", "Release Date"], axis=1)
# temp

###lyrics ovh search

In [26]:
import time
i=0
lyrics_all=[]
for idx, row in extra_songs_df.iterrows():
  i+=1
  print(i)
  song_name = row["Cleaned Title"]
  artist_name=row["Main Artist"]
  # idx%2==0: time.sleep(0.2) #slow down
  lyrics=get_lyrics(song_name, artist_name)
  if lyrics==None: print("Couln't find lyric for: ", artist_name, song_name)
  lyrics_all.append(lyrics)

print(len(lyrics_all))
extra_songs_df["Lyrics"]=lyrics_all


1
2
3
4
5
Couln't find lyric for:  Lana Del Rey A&W
6
Couln't find lyric for:  Killer Mike Scientists & Engineers
7
8
9
Couln't find lyric for:  Bonnie Raitt Just Like That
10
11
Couln't find lyric for:  Tony Bennett I Get A Kick Out Of You
12
13
14
15
Couln't find lyric for:  H.E.R. Fight For You
16
17
18
19
Couln't find lyric for:  H.E.R. I Can'T Breathe
20
Couln't find lyric for:  H.E.R. Hard Place
21
22
23
Couln't find lyric for:  H.E.R. Hard Place
24
25
Couln't find lyric for:  Ybn Cordae Bad Idea
26
27
28
29
30
31
Couln't find lyric for:  Danger Mouse Chase Me
32
33
34
35
36
37
38
39
40
41
42
42


## genius data

In [32]:
import requests


def search_song_GENIUS(song_name, artist_name):
    base_url = "https://api.genius.com/search"
    query = f"{song_name} {artist_name}"
    response = requests.get(base_url, params={'q': query}, headers=headers)

    if response.status_code != 200:
        print("Failed to search Genius")
        return None, None

    results = response.json()['response']['hits']
    for hit in results:
        if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
            url = hit['result']['url']
            song_id = hit['result']['id']
            return url, song_id

    return None, None


In [30]:
def get_song_album(song_id):
    song_url = f"https://api.genius.com/songs/{song_id}"
    response = requests.get(song_url, headers=headers)

    if response.status_code != 200:
        return None

    song_info = response.json()['response']['song']
    album = song_info.get('album')
    return album['name'] if album else None


In [31]:
def get_song_date(song_id):
    song_url = f"https://api.genius.com/songs/{song_id}"
    response = requests.get(song_url, headers=headers)

    if response.status_code != 200:
        return None

    song_info = response.json()['response']['song']
    return song_info.get('release_date', None)


In [33]:
import time
i=0
genius_found=[]
release_date=[]
album_genius=[]
for idx, row in extra_songs_df.iterrows():
  i+=1
  print(i)
  song_name = row["Cleaned Title"]
  artist_name=row["Main Artist"]
  url, id= search_song_GENIUS(song_name, artist_name)
  # if idx%2==0: time.sleep(0.2) #to avoid too many request per second
  Flag=1
  # Extract relevant details
  if(id==None):
    print("Song not found", row )

    print("-"*10)
    Flag=0
  else:  #song was found
    date= get_song_date(id)
    #extract album name of the data if it wasnt found on spotify
    if row["Spotify Found"]=='0' and row['Album']=="":
      print("Genius Found, but spotify didn't. Search for Album of:", song_name," by ", artist_name)
      album= get_song_album(id)
    else: album=None

  genius_found.append(Flag) #0 if not found else 1
  release_date.append(date if Flag else None)
  album_genius.append(album if Flag else None )

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
Song not found Song ID                                                        NaN
Title                                                     Bad Idea
Cleaned Title                                             Bad Idea
Artist                      YBN Cordae Featuring Chance The Rapper
Main Artist                                             Ybn Cordae
Feat                                             Chance The Rapper
Song Writers/Credits                                           NaN
Producer                                                       NaN
Label                                                          NaN
Album                                                 The Lost Boy
Duration                                                   234.614
Spotify Featured Artists                         Chance the Rapper
Spotify Found                                                    1
Release Date                                    

In [34]:
print(len(genius_found)) #42
print(len(release_date))
print(len(album_genius))


42
42
42


In [35]:
extra_songs_df['Genius Album']=album_genius
extra_songs_df['Release Date']=release_date
extra_songs_df['Genius Found']=genius_found

In [36]:
extra_songs_df

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,...,Spotify Found,Release Date,Genius Album,Genius Found,Lyrics,Genre,Tempo,Loudness,Cleaned Main Artist,ArtistID
0,NaN,Asteroids,Asteroids,Rapsody Featuring Hit-Boy,Rapsody,Hit-Boy,NaN,NaN,NaN,Please Don't Cry,...,1,2023-10-29,None,1,"First name Marlanna, last name Evans\r\nUndera...",Rap/Hip Hop,125.890709,0.886106,Rapsody,NaN
1,NaN,Not Strong Enough,Not Strong Enough,boygenius,Boygenius,NaN,NaN,NaN,NaN,the record,...,1,2023-03-01,None,1,Black hole opened in the kitchen\r\nEvery cloc...,Alternative,127.000000,NaN,Boygenius,NaN
2,NaN,Worship,Worship,Jon Batiste,Jon Batiste,NaN,NaN,NaN,NaN,World Music Radio,...,1,2023-08-18,None,1,We are born the same\r\nWe are born the same\r...,Jazz,97.000000,NaN,Jon Batiste,NaN
3,NaN,Butterfly,Butterfly,Jon Batiste,Jon Batiste,NaN,NaN,NaN,NaN,World Music Radio,...,1,2023-08-18,None,1,A butterfly flying home\r\nBut can you fly on ...,Pop,140.000000,NaN,Jon Batiste,NaN
4,NaN,A&W,A&W,Lana Del Rey,Lana Del Rey,NaN,NaN,NaN,NaN,Did you know that there's a tunnel under Ocean...,...,1,2023-02-14,None,1,None,Rap/Hip Hop,162.782684,0.900911,Lana Del Rey,NaN
5,NaN,SCIENTISTS & ENGINEERS,Scientists & Engineers,"Killer Mike Featuring André 3000, Future And E...",Killer Mike,"André 3000, Future And Eryn Allen Kane",NaN,NaN,NaN,MICHAEL,...,1,2023-06-13,None,1,None,Rap/Hip Hop,140.000000,NaN,Killer Mike,NaN
6,NaN,You And Me On The Rock,You And Me On The Rock,Brandi Carlile Featuring Lucius,Brandi Carlile,Lucius,NaN,NaN,NaN,In These Silent Days,...,1,2021-10-01,None,1,They build wooden houses on frozen ponds\r\nIn...,Singer & Songwriter,113.995728,0.853343,Brandi Carlile,NaN
7,NaN,Don't Shut Me Down,Don'T Shut Me Down,ABBA,Abba,NaN,NaN,NaN,NaN,I Still Have Faith In You / Don't Shut Me Down,...,1,2021-09-02,None,1,A while ago I heard the sound of children's la...,Pop,99.925713,0.637480,Abba,NaN
8,NaN,Just Like That,Just Like That,Bonnie Raitt,Bonnie Raitt,NaN,NaN,NaN,NaN,Just Like That...,...,1,2022-04-22,None,1,None,Rock,81.000000,NaN,Bonnie Raitt,NaN
9,NaN,Right On Time,Right On Time,Brandi Carlile,Brandi Carlile,NaN,NaN,NaN,NaN,Right on Time,...,1,2021-07-21,None,1,"Come back now, even if you call me out\r\nYou ...",Singer & Songwriter,126.191765,0.469051,Brandi Carlile,NaN


In [37]:
extra_songs_df=extra_songs_df.drop_duplicates()
extra_songs_df #38 x 22

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,...,Spotify Found,Release Date,Genius Album,Genius Found,Lyrics,Genre,Tempo,Loudness,Cleaned Main Artist,ArtistID
0,NaN,Asteroids,Asteroids,Rapsody Featuring Hit-Boy,Rapsody,Hit-Boy,NaN,NaN,NaN,Please Don't Cry,...,1,2023-10-29,None,1,"First name Marlanna, last name Evans\r\nUndera...",Rap/Hip Hop,125.890709,0.886106,Rapsody,NaN
1,NaN,Not Strong Enough,Not Strong Enough,boygenius,Boygenius,NaN,NaN,NaN,NaN,the record,...,1,2023-03-01,None,1,Black hole opened in the kitchen\r\nEvery cloc...,Alternative,127.000000,NaN,Boygenius,NaN
2,NaN,Worship,Worship,Jon Batiste,Jon Batiste,NaN,NaN,NaN,NaN,World Music Radio,...,1,2023-08-18,None,1,We are born the same\r\nWe are born the same\r...,Jazz,97.000000,NaN,Jon Batiste,NaN
3,NaN,Butterfly,Butterfly,Jon Batiste,Jon Batiste,NaN,NaN,NaN,NaN,World Music Radio,...,1,2023-08-18,None,1,A butterfly flying home\r\nBut can you fly on ...,Pop,140.000000,NaN,Jon Batiste,NaN
4,NaN,A&W,A&W,Lana Del Rey,Lana Del Rey,NaN,NaN,NaN,NaN,Did you know that there's a tunnel under Ocean...,...,1,2023-02-14,None,1,None,Rap/Hip Hop,162.782684,0.900911,Lana Del Rey,NaN
5,NaN,SCIENTISTS & ENGINEERS,Scientists & Engineers,"Killer Mike Featuring André 3000, Future And E...",Killer Mike,"André 3000, Future And Eryn Allen Kane",NaN,NaN,NaN,MICHAEL,...,1,2023-06-13,None,1,None,Rap/Hip Hop,140.000000,NaN,Killer Mike,NaN
6,NaN,You And Me On The Rock,You And Me On The Rock,Brandi Carlile Featuring Lucius,Brandi Carlile,Lucius,NaN,NaN,NaN,In These Silent Days,...,1,2021-10-01,None,1,They build wooden houses on frozen ponds\r\nIn...,Singer & Songwriter,113.995728,0.853343,Brandi Carlile,NaN
7,NaN,Don't Shut Me Down,Don'T Shut Me Down,ABBA,Abba,NaN,NaN,NaN,NaN,I Still Have Faith In You / Don't Shut Me Down,...,1,2021-09-02,None,1,A while ago I heard the sound of children's la...,Pop,99.925713,0.637480,Abba,NaN
8,NaN,Just Like That,Just Like That,Bonnie Raitt,Bonnie Raitt,NaN,NaN,NaN,NaN,Just Like That...,...,1,2022-04-22,None,1,None,Rock,81.000000,NaN,Bonnie Raitt,NaN
9,NaN,Right On Time,Right On Time,Brandi Carlile,Brandi Carlile,NaN,NaN,NaN,NaN,Right on Time,...,1,2021-07-21,None,1,"Come back now, even if you call me out\r\nYou ...",Singer & Songwriter,126.191765,0.469051,Brandi Carlile,NaN


In [38]:
extra_songs_df.to_csv("cleaned_missing_songs_grammy_dwi.csv")

perfect now we have the df with the extra songs and all the needed info. all we need to do is to add them to our OG df. Before that we will go get the artist ID.
